In [1]:
import pandas as pd

In [35]:
df = pd.read_csv('casos_covid19.csv', low_memory=False, index_col='numero_de_caso',
                    usecols = ['numero_de_caso', 'fecha_apertura_snvs', 'clasificacion','fecha_alta', 
                    'fecha_fallecimiento', 'tipo_contagio', 'provincia', 'edad'])

df = df.loc[df['provincia'] == 'CABA']
df = df.loc[df['edad'] >= 0]
df = df.loc[df['edad'] <= 113]

df['fecha_toma_muestra'] = pd.to_datetime(df['fecha_apertura_snvs'], format='%d%b%Y:%H:%M:%S.%f')
df['fecha_alta'] = pd.to_datetime(df['fecha_alta'], format='%d%b%Y:%H:%M:%S.%f')
df['fecha_fallecimiento'] = pd.to_datetime(df['fecha_fallecimiento'], format='%d%b%Y:%H:%M:%S.%f')
df.fecha_fallecimiento.astype("datetime64")

df.drop(['provincia', 'edad', 'fecha_apertura_snvs'], axis =1, inplace=True)

df.to_csv('db/db_casos.csv')

df.head(5)


,clasificacion,fecha_fallecimiento,fecha_alta,tipo_contagio,fecha_toma_muestra
numero_de_caso,,,,,
16371068,confirmado,NaT,NaT,Comunitario,2021-07-08
16371082,confirmado,NaT,NaT,Comunitario,2021-07-08
16371100,confirmado,NaT,NaT,Contacto,2021-07-08
16371112,confirmado,NaT,NaT,Comunitario,2021-07-08
16371128,confirmado,NaT,NaT,Comunitario,2021-07-08


In [36]:
# Filtrada para usar de base para las demas
df = pd.read_csv('casos_covid19.csv', low_memory=False, index_col='numero_de_caso')

df = df.loc[df['provincia'] == 'CABA']
df = df.loc[df['edad'] >= 0]
df = df.loc[df['edad'] <= 113]

df.to_csv('db/db_casos_base.csv')

df.head(5)

,fecha_apertura_snvs,fecha_toma_muestra,fecha_clasificacion,provincia,barrio,comuna,genero,edad,clasificacion,fecha_fallecimiento,fallecido,fecha_alta,tipo_contagio
numero_de_caso,,,,,,,,,,,,,
16371068,08JUL2021:00:00:00.000000,08JUL2021:00:00:00.000000,08JUL2021:00:00:00.000000,CABA,VILLA DEVOTO,11.0,femenino,39.0,confirmado,NaN,NaN,NaN,Comunitario
16371082,08JUL2021:00:00:00.000000,08JUL2021:00:00:00.000000,08JUL2021:00:00:00.000000,CABA,VILLA PUEYRREDON,12.0,femenino,31.0,confirmado,NaN,NaN,NaN,Comunitario
16371100,08JUL2021:00:00:00.000000,08JUL2021:00:00:00.000000,08JUL2021:00:00:00.000000,CABA,NaN,NaN,femenino,28.0,confirmado,NaN,NaN,NaN,Contacto
16371112,08JUL2021:00:00:00.000000,08JUL2021:00:00:00.000000,08JUL2021:00:00:00.000000,CABA,CONSTITUCION,1.0,femenino,51.0,confirmado,NaN,NaN,NaN,Comunitario
16371128,08JUL2021:00:00:00.000000,08JUL2021:00:00:00.000000,08JUL2021:00:00:00.000000,CABA,PALERMO,14.0,masculino,34.0,confirmado,NaN,NaN,NaN,Comunitario


In [37]:
# BARRIOS
df = pd.read_csv('db/db_casos_base.csv', low_memory=False,
                    usecols = ['barrio', 'provincia', 'comuna'])

df['comuna'].fillna(value='N/I', inplace=True)
df['barrio'].fillna(value='N/I', inplace=True)
df['provincia'].fillna(value='N/I', inplace=True)


df.groupby(by = ["barrio", "comuna", "provincia"])
df = df.drop_duplicates()
df.to_csv('db/db_barrios_sinid.csv')

In [38]:
df = pd.read_csv('db/db_barrios_sinid.csv', low_memory=False, 
                    usecols= ['provincia', 'barrio', 'comuna'])
                    
df['barrio'].fillna(value='N/I', inplace=True)     
df.index.names = ['id_barrio']          
df.to_csv('db/db_barrios.csv')

In [39]:
# PERSONAS
dfp = pd.read_csv('db/db_casos_base.csv', low_memory=False,
                    usecols = ['numero_de_caso', 'barrio', 'genero', 'edad'])

dfp['barrio'].fillna(value='N/I', inplace=True)
dfp['edad'].fillna(value='1', inplace=True)
dfp['edad'].astype("int64")

personas = dfp.copy()
personas.head()


,numero_de_caso,barrio,genero,edad
0,16371068,VILLA DEVOTO,femenino,39.0
1,16371082,VILLA PUEYRREDON,femenino,31.0
2,16371100,N/I,femenino,28.0
3,16371112,CONSTITUCION,femenino,51.0
4,16371128,PALERMO,masculino,34.0


In [40]:
dfb = pd.read_csv('db/db_barrios.csv', low_memory=False)
dfb.head()

,id_barrio,provincia,barrio,comuna
0,0,CABA,VILLA DEVOTO,11.0
1,1,CABA,VILLA PUEYRREDON,12.0
2,2,CABA,N/I,N/I
3,3,CABA,CONSTITUCION,1.0
4,4,CABA,PALERMO,14.0


In [41]:

merged = dfb.merge(personas, how='inner', on='barrio')
merged = merged.drop(['barrio', 'comuna', 'provincia'], axis=1)

merged.head()
merged.to_csv('db/db_personas.csv', index=False)
